In [4]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as integrate
from numba import njit

In [2]:
import matplotlib
matplotlib.get_backend()

'module://matplotlib_inline.backend_inline'

try with generators https://realpython.com/introduction-to-python-generators/

normalizing only works for linear homog recurrence relation.

get recurrence relation.

numerical integration gets computationally expensive bc a lot of n, alpha, x combinations

so use recurrence. know K0 analytically from integral bc simple to evaluate. 

recurrence relation is linear but not homogeneous. could maybe guess particular solution so that we can use bessel function method.

$\alpha / z$ sets critical value for when to use upwards vs downwards recurrence. set this critical value at 1/2. could reduce this number.

for downward recurrence, use power series to get Kn,max. this is expensive, but it allows us to get many others in the recurrence relation for cheap. cost is upfront. coefficients in power series are small. could use numpy integrator but with this way we can control accuracy by adding more terms. 

there is an analytic solution in terms of a power series. we can use this to check.

can also use scipy integrate to check work.

for homework, calculate n=0 to 10. and for several alpha/z

plot Ks together.

fxn of z alpha n
set epsilon for accuracy. if alpha / z ^ k < epsilon, terminate sum. 
set 1/2 for critical down/up.
if less down
    - taylor expand for KN (large N), then go down. keep going until abs value is small. xn=1 ; for m in large number (x_n / z)/(n+m+1). just a taylor series
    - then can just do downward recursion

if greater up: simple. return Kn

plot for all n from 1 to 10
compare different alpha/z values plotting might be weird bc one might be much larger than others
alpha/z = 0.1 0.2 0.3 0.5



In [64]:
class KnSolver:
    def __init__(self, alpha, z, n_max, epsilon):
        self._alpha = alpha
        self._z = z
        self._epsilon = epsilon
        self._n_max = n_max
        self._critical_ratio = 0.5
        
    def calculate_K_zero(self):
        return (1 / self._alpha) * np.log(1 + self._alpha / self._z)
    
    def _calculate_K_zero_at_z(self, z):
        return (1 / self._alpha) * np.log(1 + self._alpha / z)
    
    #@staticmethod
    #@njit
    def analytic_solution_Kn_at_z(self, n, z):
        
        k = 1
        coefficient = 1
        sum = 0
        max_iter = 100000
        while np.abs(coefficient) > self._epsilon:
            coefficient = (-self._alpha / z)**k 
            sum += coefficient / k
            k += 1
            
            if k > max_iter:
                print(f'Warning: exceeding {max_iter} iterations.')
                break
        print(f'Number of iterations: {k}')
        print(sum)
        print(coefficient)

        return (-z/self._alpha)**n * (self._calculate_K_zero_at_z(z) + sum / self._alpha)
    
    
    def upwards_recurrence(self):
        Kn = self.calculate_K_zero()
        
        Kn_history = np.zeros((self._n_max, len(self._z)))
        
        for n in range(self._n_max):
            Kn_history[n, :] = Kn
            Kn = 1/(self._alpha * (n + 1)) - (self._z / self._alpha) * Kn
        
        return Kn_history
    
    def downwards_recurrence(self):
        Kn_max = np.array([self.analytic_solution_Kn_at_z(self._n_max, z) for z in self._z])
        Kn_history = np.zeros((self._n_max + 1, len(self._z)))
        Kn_history[-1, :] = Kn_max
        print(Kn_history)
        
        Kn = Kn_max
        for n in range(self._n_max - 1, -1, -1):
            print(f'fK{n} is {Kn}')
            Kn = (-self._alpha / self._z) * (Kn - 1/(self._alpha * (n + 1)))
            Kn_history[self._n_max - n, :] = Kn

        return Kn_history
    
    
    def solve_Kn_integral(self):
        def integrand(x, z, alpha, n):
            return x**n / (z + alpha * x)
        
        Kn_history = np.zeros((self._n_max, len(self._z)))
    
        for n in range(self._n_max):
            for i, z in enumerate(self._z):    
                Kn_history[n, i] = integrate.quad(integrand, 0, 1, args=(z, self._alpha, n))[0]
        
        return Kn_history
    
    
    def taylor_approximate_Kn(self, n, z):
        
        coefficient = 1
        k = 0
        sum = 0
        max_iter = 100000
        while np.abs(coefficient) > self._epsilon:
            coefficient = (-self._alpha / z)**k 
            sum += coefficient / (n + k + 1)
            k += 1 
            
            if k > max_iter:
                print(f'Warning: exceeding {max_iter} iterations.')
                break
        print(f'Number of iterations: {k}')
        return sum / z
    
    def solve_analytically(self):
        #Kn_history = np.zeros((self._n_max, len(self._z)))
        Kn_history = np.array([[self.analytic_solution_Kn_at_z(n, z) for z in self._z] for n in range(self._n_max)])
        
        return Kn_history
        
    
    def calculate_Kn(self):
        
        # if alpha /z down
        # if alopha / up
        
        
        
        #if np.abs(self._alpha / self._z)
        
        raise NotImplementedError
        
    

In [65]:
1e-200

1e-200

In [66]:
#solver = KnSolver(0.1, np.array([1, 2, 3, 4]), 10, 1e-2)
solver = KnSolver(1e-4, np.array([1, 2, 3, 4]), 10, 1e-2)


In [67]:
hist = test = solver.solve_analytically()

Number of iterations: 2
-0.0001
-0.0001
Number of iterations: 2
-5e-05
-5e-05
Number of iterations: 2
-3.3333333333333335e-05
-3.3333333333333335e-05
Number of iterations: 2
-2.5e-05
-2.5e-05
Number of iterations: 2
-0.0001
-0.0001
Number of iterations: 2
-5e-05
-5e-05
Number of iterations: 2
-3.3333333333333335e-05
-3.3333333333333335e-05
Number of iterations: 2
-2.5e-05
-2.5e-05
Number of iterations: 2
-0.0001
-0.0001
Number of iterations: 2
-5e-05
-5e-05
Number of iterations: 2
-3.3333333333333335e-05
-3.3333333333333335e-05
Number of iterations: 2
-2.5e-05
-2.5e-05
Number of iterations: 2
-0.0001
-0.0001
Number of iterations: 2
-5e-05
-5e-05
Number of iterations: 2
-3.3333333333333335e-05
-3.3333333333333335e-05
Number of iterations: 2
-2.5e-05
-2.5e-05
Number of iterations: 2
-0.0001
-0.0001
Number of iterations: 2
-5e-05
-5e-05
Number of iterations: 2
-3.3333333333333335e-05
-3.3333333333333335e-05
Number of iterations: 2
-2.5e-05
-2.5e-05
Number of iterations: 2
-0.0001
-0.0001


In [68]:
hist

array([[-4.99966670e-05, -1.24995823e-05, -5.55543214e-06,
        -3.12494850e-06],
       [ 4.99966670e-01,  2.49991646e-01,  1.66662964e-01,
         1.24997940e-01],
       [-4.99966670e+03, -4.99983292e+03, -4.99988892e+03,
        -4.99991760e+03],
       [ 4.99966670e+07,  9.99966584e+07,  1.49996668e+08,
         1.99996704e+08],
       [-4.99966670e+11, -1.99993317e+12, -4.49990003e+12,
        -7.99986816e+12],
       [ 4.99966670e+15,  3.99986633e+16,  1.34997001e+17,
         3.19994726e+17],
       [-4.99966670e+19, -7.99973267e+20, -4.04991003e+21,
        -1.27997891e+22],
       [ 4.99966670e+23,  1.59994653e+25,  1.21497301e+26,
         5.11991562e+26],
       [-4.99966670e+27, -3.19989307e+29, -3.64491903e+30,
        -2.04796625e+31],
       [ 4.99966670e+31,  6.39978613e+33,  1.09347571e+35,
         8.19186500e+35]])

In [69]:
test

array([[-4.99966670e-05, -1.24995823e-05, -5.55543214e-06,
        -3.12494850e-06],
       [ 4.99966670e-01,  2.49991646e-01,  1.66662964e-01,
         1.24997940e-01],
       [-4.99966670e+03, -4.99983292e+03, -4.99988892e+03,
        -4.99991760e+03],
       [ 4.99966670e+07,  9.99966584e+07,  1.49996668e+08,
         1.99996704e+08],
       [-4.99966670e+11, -1.99993317e+12, -4.49990003e+12,
        -7.99986816e+12],
       [ 4.99966670e+15,  3.99986633e+16,  1.34997001e+17,
         3.19994726e+17],
       [-4.99966670e+19, -7.99973267e+20, -4.04991003e+21,
        -1.27997891e+22],
       [ 4.99966670e+23,  1.59994653e+25,  1.21497301e+26,
         5.11991562e+26],
       [-4.99966670e+27, -3.19989307e+29, -3.64491903e+30,
        -2.04796625e+31],
       [ 4.99966670e+31,  6.39978613e+33,  1.09347571e+35,
         8.19186500e+35]])

In [70]:
solver.taylor_approximate_Kn(2, 4)

Number of iterations: 2


0.08333177083333333

In [72]:
solver.analytic_solution_Kn_at_z(2, 4)

Number of iterations: 2
-2.5e-05
-2.5e-05


-4999.9176004345

In [60]:
Kn_history = solver.upwards_recurrence()

In [61]:
print(Kn_history)

[[ 9.99950003e-01  4.99987500e-01  3.33327778e-01  2.49996875e-01]
 [ 4.99966670e-01  2.49991646e-01  1.66662964e-01  1.24997940e-01]
 [ 3.33297321e-01  1.67082435e-01  1.11075296e-01  8.23995651e-02]
 [ 3.60122843e-01 -8.31536522e+00  1.07443998e+00  3.73507276e+01]
 [-1.10122843e+03  1.68807304e+05 -2.97331995e+04 -1.49152910e+06]
 [ 1.10142843e+07 -3.37614409e+09  8.91997984e+08  5.96611661e+10]
 [-1.10142841e+11  6.75228817e+13 -2.67599395e+13 -2.38644664e+15]
 [ 1.10142841e+15 -1.35045763e+18  8.02798185e+17  9.54578657e+19]
 [-1.10142841e+19  2.70091527e+22 -2.40839456e+22 -3.81831463e+24]
 [ 1.10142841e+23 -5.40183054e+26  7.22518367e+26  1.52732585e+29]]


In [62]:
Kn_history_down = solver.downwards_recurrence()

AttributeError: 'KnSolver' object has no attribute 'analytic_solution_Kn'

In [63]:
Kn_history_down

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 4.99966670e+31,  6.39978613e+33,  1.09347571e+35,
         8.19186500e+35],
       [-4.99966670e+27, -3.19989307e+29, -3.64491903e+30,
        -2.04796625e+31],
       [ 4.99966670e+23,  1.59994653e+25,  1.21497301e+26,
         5.11991562e+26],
       [-4.99966670e+19, -7.99973267e+20, -4.04991003e+21,
        -1.27997891e+22],
       [ 4.99966670e+15,  3.99986633e+16,  1.34997001e+17,
         3.19994726e+17],
       [-4.99966670e+11, -1.99993317e+12, -4.49990003e+12,
        -7.99986816e+12],
       [ 4.99966673e+07,  9.99966585e+07,  1.49996668e+08,
         1.99996704e+08],
       [-4.99933339e+03, -4.99966626e+03, -4.99977782e+03,
        -4.99983427e+03],
       [ 9.99933339e-01,  4.99983313e-01,  3.33325927e-01,
         2.49995857e-01],
       [ 9.99900007e-01,  4.99975001e-01,  3.33322222e-01,
         2.49993750e-01]])

In [31]:
solver.solve_Kn_integral()

array([[0.99995   , 0.4999875 , 0.33332778, 0.24999688],
       [0.49996667, 0.24999167, 0.16666296, 0.12499792],
       [0.33330834, 0.16666042, 0.11110833, 0.08333177],
       [0.24998   , 0.124995  , 0.08333111, 0.06249875],
       [0.19998333, 0.09999583, 0.06666481, 0.04999896],
       [0.16665238, 0.08332976, 0.05555397, 0.04166577],
       [0.14284464, 0.07142545, 0.04761766, 0.0357135 ],
       [0.12498889, 0.06249722, 0.04166543, 0.03124931],
       [0.11110111, 0.05555306, 0.03703593, 0.02777715],
       [0.09999091, 0.04999773, 0.03333232, 0.02499943]])